このファイルについて
- about
    - 検索履歴データから館山道・関越道の各区間の5分単位の検索数を計算する
- author: 松永
- Input
    - ../Input_processed_data/search_records/csv202xxx/*
- Output
    - 時間指定あり検索
        - ../Input_processed_data/search_count/search-count_{tateyama or kannetsu}.csv
    - 時間指定なし検索
        - ../Input_processed_data/search_count/search-count_{tateyama or kannetsu}_unspecified.csv"

In [1]:
import datetime as dt
import os
import pickle
import time
from typing import Dict, List, Optional, Set, Tuple

import networkx as nx
import numpy as np
import pandas as pd
import cudf
# import warnings
# warnings.simplefilter('ignore')

In [2]:
# data directory
DATA_DIR = '../../Input_processed_data'

# IC, 道路情報 csv
IC_CSV = f'{DATA_DIR}/road_master/ic_preprocessed.csv'
IC_NET_CSV = f'{DATA_DIR}/road_master/220303-doronet_ic.csv'
IC_SUBNET_CSV = f'{DATA_DIR}/road_master/icnet_sub.csv'

# 検索ログ csv
SEARCH_LOG_DIR = lambda month: f'{DATA_DIR}/simple_search_records/csv{month}'
SEARCH_LOG_CSV = lambda date: f'{SEARCH_LOG_DIR(date[:6])}/record_{date}.csv'

# 準備

## 道路構造データの読み込み

In [3]:
# モジュール内で前処理済み
df_ic = pd.read_csv(IC_CSV, dtype={'ic_code': str})
df_icnet = pd.read_csv(IC_NET_CSV, dtype={'start_code': str, 'end_code': str, 'road_code': str})
sub_icnet = pd.read_csv(IC_SUBNET_CSV, dtype={'start_code': str, 'end_code': str, 'road_code': str})

code2name = dict(zip(df_ic['ic_code'], df_ic['ic_name']))
name2code = {v: k for k, v in code2name.items()}

ic_graph = nx.from_pandas_edgelist(
    df_icnet, source='start_code', target='end_code',
    edge_attr=['distance', 'road_code', 'direction'], create_using=nx.DiGraph())

In [4]:
# 区間ごとの制限速度を格納したテーブル, Map を作成
df_limits = sub_icnet.loc[:, ['start_code', 'end_code', 'start_name', 'end_name', 'road_code', 'limit']]
limit_dict = {
    (s_code, e_code): lim for s_code, e_code, lim in df_limits.loc[:, ['start_code', 'end_code', 'limit']].values
}

In [5]:
ic_nodes_set: set = set(code2name.keys())

In [6]:
# トラカンデータが持つ区間のdataframe
tc_segments = pd.read_pickle(f'./traffic_counter_segments.pkl')
tc_segments_set = set(
    [tuple(segment) for segment in tc_segments.loc[:, ['start_code', 'end_code']].values]
)

## 最短経路オブジェクト`route_dict`の読み込み・書き出し

In [7]:
fname = './route_dict.pkl'

if os.path.exists(fname): # 経路マップがすでに存在しているとき、それを使う
    with open(fname, 'rb') as f:
        print('Loading IC Routes...')
        route_dict = pickle.load(f)
else: # 存在していなければ計算してバイナリで保存
    print('Calculating IC Routes...')
    route_dict = dict(nx.all_pairs_dijkstra_path(ic_graph, weight='distance'))
    
    with open(fname, 'wb') as f:
        pickle.dump(route_dict, f)
print('Finished.')

Loading IC Routes...
Finished.


In [8]:
! du -h ./route_dict.pkl

3.4G	./route_dict.pkl


## 省略するICの集合を読み込む

In [9]:
with open('small_ic.pkl', 'rb') as f:
    df_small_ic = pickle.load(f)

small_ic_set = set(df_small_ic.ic_code)

# Functions

## 経路計算

In [10]:
def __get_route(
    src: str, dest: str, route_dict: Dict[str, Dict[str, List[str]]]
) -> Optional[List[str]]:
    if not (src in ic_nodes_set and dest in ic_nodes_set):
        return []
    try:
        path = route_dict[src][dest]
        return path
    except: # 経路が存在しない, もしくはノードがグラフ上に存在しない場合
        return []

In [11]:
def get_route(
    src: str, 
    dest: str, 
    route_dict: Dict[str, Dict[str, List[str]]],
    excluded_ic_set: Set[str] = set(),
) -> List[str]:
    '''
    ic_graph上で出発地から目的地までの経路を得る関数

    Parameters
    --------------
    src: 出発ICコード
    dest: 目的ICコード
    '''
    path = __get_route(src, dest, route_dict=route_dict)
    
    if len(excluded_ic_set) > 0:
        path = [ic for ic in path if ic not in excluded_ic_set]
    return path

In [12]:
def get_route_with_time(
    src: str,
    dest: str,
    spec_datetime: dt.datetime,
    spec_type: int,
    route_dict: Dict[str, Dict[str, List[str]]],
    excluded_ic_set: Set[str] = set(),
) -> List[Tuple[str, dt.datetime]]:
    '''
    ic_graph上で出発地（src_name）から目的地（target_name）までの予想通過時刻付き経路を得る関数
    '''
    # 関越道・館山道 以外の道路の移動速度は80km/hと仮定する
    DEFAULT_SPEED = 80
    
    path = __get_route(src, dest, route_dict=route_dict)

    elapsed = dt.timedelta()
    elapsed_time_list = [elapsed]
    for i, (start, end) in enumerate(zip(path, path[1:])):
        dist = ic_graph[start][end]['distance']
        limit_speed = limit_dict.get((start, end), DEFAULT_SPEED)
        
        # 所要時間を積算
        td = dt.timedelta(hours = dist / limit_speed)
        elapsed += td
        elapsed_time_list.append(elapsed)
    
    if spec_type == 1:
        time_list = [spec_datetime + td for td in elapsed_time_list]
    elif spec_type == 2:
        time_list = [spec_datetime - td for td in elapsed_time_list[::-1]]
    else:
        time_list = [spec_datetime + td for td in elapsed_time_list]
    
    result = list(zip(path, time_list))
    if len(excluded_ic_set) > 0:
        result = [(ic, time) for (ic, time) in result if ic not in excluded_ic_set]
    return result

## 検索ログの読み込み

In [13]:
def str2date(date: str, format: str = '%Y%m%d') -> dt.date:
    return dt.datetime.strptime(date, '%Y%m%d').date()

In [14]:
def get_log(date: str) -> cudf.DataFrame:
    type_map = {
        'datetime': np.datetime64,
        'start_code': 'category',
        'end_code': 'category',
        'spec_datetime': np.datetime64,
        'spec_type': 'category',
        'car_type': 'category'
    }
    if not os.path.exists(SEARCH_LOG_CSV(date)):
        return cudf.DataFrame(columns=type_map.keys())

    df = pd.read_csv(SEARCH_LOG_CSV(date)).astype(type_map)
    df = df.loc[
        (df.start_code.isin(ic_nodes_set)) &
        (df.end_code.isin(ic_nodes_set))
    ]
    return cudf.from_pandas(df)

In [15]:
def get_past_logs(date: str, periods: int, include_target: bool = False) -> cudf.DataFrame:
    '''
    指定日(target_date)から過去数日分の検索履歴データを取得する関数

    Parameters
    --------------
    date: 混雑度算出の対象となる日付（文字列 or リスト)
    periods: 過去何日分の履歴を参照するか
    include_target: dateをデータに含めるかどうか
    '''
    # 参照すべき全日付のiterableを生成
    if include_target:
        dt_range = pd.date_range(end=date, periods=periods)
    else:
        dt_range = pd.date_range(end=date, periods=periods+1, closed='left')

    DAYS = [d.strftime('%Y%m%d') for d in dt_range]

    df_specified = cudf.DataFrame()
    for d in DAYS:
        df = get_log(d)
        if len(df) == 0:
            continue
        
        series_spec_date: pd.Series = df.spec_datetime.to_pandas().map(lambda dt: dt.date())
        _df_specified = df.loc[series_spec_date == str2date(date)]
        df_specified = cudf.concat([df_specified, _df_specified], ignore_index=True)
    df_specified.reset_index(drop=True, inplace=True)
    
    if len(df_specified) == 0:
        df_specified = cudf.DataFrame(columns=df.columns)
    return df_specified

In [16]:
def get_unspecified_log(date: str) -> cudf.DataFrame:
    '''
    検索履歴全体から時間指定なしの検索のみを得る関数
    '''
    ALLOWED_MINUTE_LAG = 15
    
    df = get_log(date)
    # 検索日時と指定日時の差を秒単位で算出
    t_diff = abs((df.datetime - df.spec_datetime) / np.timedelta64(1, 's'))
    
    # 事前に定義した時間差（=15分）より検索日時と指定日時の差が少ないレコードのみを抽出
    df_unspecified = df.loc[t_diff < ALLOWED_MINUTE_LAG * 60].reset_index(drop=True)
    return df_unspecified

## 検索ログのマッピング

In [17]:
def map_search_to_road_network(
    df: cudf.DataFrame,
    route_dict: Dict[str, Dict[str, List[str]]],
    excluded_ic_set: Set[str] = set()
) -> cudf.DataFrame:
    '''
    検索ログから各道路区間の予想通過時刻を計算する
    '''
    start_code_list = []
    end_code_list = []
    passing_time_list = []
    
    columns = ['start_code', 'end_code', 'spec_datetime', 'spec_type']
    for (src, dest, spec_datetime, spec_type) in df.loc[:, columns].to_pandas().values:
        path = get_route_with_time(
            src, dest, spec_datetime, spec_type, 
            route_dict=route_dict, 
            excluded_ic_set=excluded_ic_set
        )
        for i, (start, end) in enumerate(zip(path, path[1:])):
            s_code, e_code = start[0], end[0]
            s_ptime = start[-1]
            
            if ic_graph[s_code][e_code]['road_code'] not in target_road_code_set:
                break
            
            start_code_list.append(s_code)
            end_code_list.append(e_code)
            passing_time_list.append(s_ptime)
            
        if spec_type not in {1, 2} and len(path) > 0:
            with open('./error_spec_type_records.csv', 'a') as f:
                print(f'{s_ptime},{src},{dest},{spec_datetime},{spec_type}', file=f)
                


    return cudf.DataFrame({
        'start_code': start_code_list,
        'end_code': end_code_list,
        'passing_time': passing_time_list,
    }).astype({
        'start_code': 'category',
        'end_code': 'category'
    })

In [18]:
def aggregate_search_count(df: cudf.DataFrame, timeslice: str = '5min') -> cudf.DataFrame:
    # 検索量として積算される対象となるカラムを付与
    result = df.assign(search=1)
    
    # timesliceでサンプリングし、検索量の和を取る
    result = (result
              .set_index('passing_time')
              .to_pandas()
              .groupby(['start_code', 'end_code'])
              .apply(lambda g: g['search'].resample(timeslice).sum())
              .reset_index())
    return cudf.from_pandas(result)

# 検索数の作成

## 時間指定あり

In [3]:
period_blocks = [
    # ('20210402', '20210630'),
    # ('20210701', '20210930'),
    # ('20211001', '20211231'),
    # ('20220101', '20220331'),
    # ('20220401', '20220630'),
    # ('20220701', '20220930'),
    # ('20221001', '20221231'),
    # ('20230101', '20230331'),
    # ('20230401', '20230630'),
    # ('20230701', '20230930'),
    # ('20231001', '20231231'),
    # ('20240101', '20240331'),
    # ('20240401', '20240506'),
    ('20240507', '20240831'),
]

In [20]:
past_periods = 7
timeslice = '5min'

In [22]:
for start_date, end_date in period_blocks:
    print('='*40, f'{start_date} -> {end_date}', '='*40)
    
    DAYS: List[str] = [
        d.strftime('%Y%m%d') for d in pd.date_range(start_date, end_date, freq='1D')
    ]
    
    df_search = cudf.DataFrame()
    for i, date in enumerate(DAYS):
        s = time.time()
        
        df = get_past_logs(date, periods=past_periods)
        
        df_mapped = map_search_to_road_network(
            df, route_dict, excluded_ic_set=small_ic_set
        )
        df_mapped = df_mapped.loc[
            df_mapped.loc[:, ['start_code', 'end_code']].to_pandas().apply(
                lambda segment: tuple(segment) in tc_segments_set, axis=1
            )
        ].reset_index(drop=True)
        
        _df_search = aggregate_search_count(df_mapped, timeslice)
        _df_search = cudf.from_pandas(
            _df_search.set_index('passing_time').to_pandas().loc[date]
        ).reset_index()

        print(f'{date} | {len(_df_search)} records ({time.time() - s:.3f} sec)')

        df_search = cudf.concat([df_search, _df_search], ignore_index=True)

    df_search.reset_index(inplace=True, drop=True)
    OUTPUT_FILE = f'./search_count/search-count_specified_{start_date}-{end_date}.csv'
    df_search.to_pandas().to_csv(OUTPUT_FILE, index=False)

======================================== 20231001 -> 20231231 ========================================
20231001 | 23040 records (44.858 sec)
20231002 | 22998 records (28.883 sec)
20231003 | 23039 records (21.484 sec)
20231004 | 22938 records (25.599 sec)
20231005 | 23003 records (30.324 sec)
20231006 | 23040 records (52.718 sec)
20231007 | 23040 records (92.813 sec)
20231008 | 23013 records (61.426 sec)
20231009 | 23000 records (41.731 sec)
20231010 | 23040 records (34.777 sec)
20231011 | 23032 records (31.604 sec)
20231012 | 23040 records (25.117 sec)
20231013 | 23040 records (43.544 sec)
20231014 | 23040 records (62.643 sec)
20231015 | 23025 records (40.475 sec)
20231016 | 23040 records (42.361 sec)
20231017 | 23039 records (23.844 sec)
20231018 | 23034 records (29.595 sec)
20231019 | 23011 records (28.092 sec)
20231020 | 23040 records (38.007 sec)
20231021 | 23040 records (71.533 sec)
20231022 | 23039 records (49.722 sec)
20231023 | 23038 records (33.119 sec)
20231024 | 23040 record

In [23]:
!head -n5 "$OUTPUT_FILE"

passing_time,start_code,end_code,search
2023-10-01 00:00:00,1040013,1040016,24
2023-10-01 00:05:00,1040013,1040016,2
2023-10-01 00:10:00,1040013,1040016,1
2023-10-01 00:15:00,1040013,1040016,3


In [25]:
!tail -n5 "$OUTPUT_FILE"

2023-12-31 23:35:00,1800106,1800111,0
2023-12-31 23:40:00,1800106,1800111,0
2023-12-31 23:45:00,1800106,1800111,0
2023-12-31 23:50:00,1800106,1800111,0
2023-12-31 23:55:00,1800106,1800111,0


## 時間指定なし

In [22]:
period_blocks = [
    # ('20210402', '20210630'),
    # ('20210701', '20210930'),
    # ('20211001', '20211231'),
    # ('20220101', '20220331'),
    # ('20220401', '20220630'),
    # ('20220701', '20220930'),
    # ('20221001', '20221231'),
    # ('20230101', '20230331'),
    # ('20230401', '20230630'),
    # ('20230701', '20230930'),
    # ('20231001', '20231231'),
    # ('20240101', '20240331'),
    # ('20240401', '20240506'),
    ()
]

In [23]:
def aggregate_search_count(df: cudf.DataFrame) -> cudf.DataFrame:
    # 検索量として積算される対象となるカラムを付与
    result = df.assign(search=1)
    
    # timesliceでサンプリングし、検索量の和を取る
    result = (result
              .set_index('passing_time')
              .to_pandas()
              .groupby(['start_code', 'end_code'])
              .apply(lambda g: g['search'].sum())
              .reset_index()
              .rename(columns={0: 'search'}))
    return cudf.from_pandas(result)

In [24]:
def shift_unspecified_search_count(df: pd.DataFrame) -> pd.DataFrame:
    '''
    時間指定なし検索を1日後ろにシフトする
    '''
    SHIFT_DAYS = 1
    
    result = (df.set_index('search_date')
              .groupby(['start_code', 'end_code'])
              .apply(lambda g: g.search.shift(SHIFT_DAYS))
               # shift後にpivot_tableっぽくなってしまう（通常はpd.Series）ため、特別に処理
              .pipe(lambda _df: _df.stack().rename('search') if not isinstance(_df, pd.Series) else _df)
              .reset_index()
              .sort_values(['search_date', 'start_code', 'end_code'])
              .reset_index(drop=True)
              .loc[:, ['search_date', 'start_code', 'end_code', 'search']])
    return result

In [25]:
def generate_date_list_from_1day_before(start_date, end_date):
    '''
    start_dateの1日前からend_dateまでの日付リストを生成する
    '''
    _start = pd.to_datetime(start_date) - pd.Timedelta(days=1)
    _end = pd.to_datetime(end_date)
    
    date_list: List[str] = [
        date.strftime('%Y%m%d') for date in pd.date_range(_start, _end, freq='1D')
    ]
    return date_list

In [26]:
for start_date, end_date in period_blocks:
    print('='*40, f'{start_date} -> {end_date}', '='*40)

    DAYS = generate_date_list_from_1day_before(start_date, end_date)
    
    df_search = cudf.DataFrame()
    for i, date in enumerate(DAYS):
        s = time.time()
        
        df = get_unspecified_log(date)
        
        df_mapped = map_search_to_road_network(
            df, route_dict, excluded_ic_set=small_ic_set
        )
        df_mapped = df_mapped.loc[
            df_mapped.loc[:, ['start_code', 'end_code']].to_pandas().apply(
                lambda segment: tuple(segment) in tc_segments_set, axis=1
            )
        ].reset_index(drop=True)
        
        _df_search = aggregate_search_count(df_mapped)
        _df_search = (_df_search
                      .assign(search_date=str2date(date).strftime('%Y-%m-%d'))
                      .loc[:, ['search_date', 'start_code', 'end_code', 'search']])

        print(f'{date} [{time.time() - s:.3f} sec]')

        df_search = cudf.concat([df_search, _df_search], ignore_index=True)
    
    df_search_shifted = shift_unspecified_search_count(df_search.to_pandas())

    OUTPUT_FILE = f'./search_count/search-count_unspecified_{start_date}-{end_date}.csv'
    df_search_shifted.to_csv(OUTPUT_FILE, index=False)

======================================== 20231001 -> 20231231 ========================================
20230930 [251.880 sec]
20231001 [157.561 sec]
20231002 [124.572 sec]
20231003 [173.768 sec]
20231004 [214.526 sec]
20231005 [170.424 sec]
20231006 [136.277 sec]
20231007 [124.961 sec]
20231008 [130.987 sec]
20231009 [130.516 sec]
20231010 [137.944 sec]
20231011 [113.512 sec]
20231012 [73.810 sec]
20231013 [112.181 sec]
20231014 [129.665 sec]
20231015 [123.401 sec]
20231016 [132.482 sec]
20231017 [116.800 sec]
20231018 [123.116 sec]
20231019 [123.924 sec]
20231020 [120.303 sec]
20231021 [111.830 sec]
20231022 [112.341 sec]
20231023 [111.482 sec]
20231024 [123.755 sec]
20231025 [111.501 sec]
20231026 [99.570 sec]
20231027 [110.472 sec]
20231028 [115.440 sec]
20231029 [119.974 sec]
20231031 [152.805 sec]
20231101 [141.833 sec]
20231102 [134.734 sec]
20231103 [123.683 sec]
20231104 [112.474 sec]
20231105 [113.433 sec]
20231106 [145.247 sec]
20231107 [126.319 sec]
20231108 [130.744 sec]
20

In [27]:
!head -n5 "$OUTPUT_FILE"

search_date,start_code,end_code,search
2024-04-01,1040013,1040016,5344.0
2024-04-01,1040016,1040013,5032.0
2024-04-01,1040016,1040020,6154.0
2024-04-01,1040020,1040016,4996.0


In [28]:
!tail -n5 "$OUTPUT_FILE"

2024-05-06,1800091,1800096,4120.0
2024-05-06,1800096,1800091,3638.0
2024-05-06,1800096,1800106,4069.0
2024-05-06,1800106,1800096,3621.0
2024-05-06,1800106,1800111,4021.0


# 好きなように期間を分割する

期間のとり方
- 2021/04/01 〜 2022/03/31
- 2022/04/01 〜 2023/03/31
- 2023/04/01 〜 2023/09/30
- 2023/10/01 ~ 2024/03/31（推論補助用）
- 2024/04/01 ~ 2024/05/06（最終評価用）

## 時間指定あり

In [31]:
dtype = {'start_code': 'category', 'end_code': 'category'}

# 時間指定あり
# df1 = pd.read_csv(f'./search_count/search-count_specified_20231001-20231231.csv', dtype=dtype)
# df2 = pd.read_csv(f'./search_count/search-count_specified_20240101-20240331.csv', dtype=dtype)

# df1.passing_time = pd.to_datetime(df1.passing_time)
# df2.passing_time = pd.to_datetime(df2.passing_time)

# 時間指定なし
df1 = pd.read_csv(f'./search_count/search-count_unspecified_20231001-20231231.csv', dtype=dtype)
df2 = pd.read_csv(f'./search_count/search-count_unspecified_20240101-20240331.csv', dtype=dtype)

In [32]:
df1.head(3)

,search_date,start_code,end_code,search
0,2023-10-01,1040013,1040016,9381.0
1,2023-10-01,1040016,1040013,8807.0
2,2023-10-01,1040016,1040020,9988.0


In [33]:
df1.tail(3)

,search_date,start_code,end_code,search
7357,2023-12-31,1800096,1800106,2938.0
7358,2023-12-31,1800106,1800096,1692.0
7359,2023-12-31,1800106,1800111,2890.0


In [34]:
df2.head(3)

,search_date,start_code,end_code,search
0,2024-01-01,1040013,1040016,4997.0
1,2024-01-01,1040016,1040013,4824.0
2,2024-01-01,1040016,1040020,5703.0


In [35]:
df2.tail(3)

,search_date,start_code,end_code,search
7277,2024-03-31,1800096,1800106,3200.0
7278,2024-03-31,1800106,1800096,2148.0
7279,2024-03-31,1800106,1800111,3148.0


In [36]:
# 2つのデータフレームを1つに結合する
df_new = pd.concat([df1, df2], axis=0)

In [37]:
df_new.head()

,search_date,start_code,end_code,search
0,2023-10-01,1040013,1040016,9381.0
1,2023-10-01,1040016,1040013,8807.0
2,2023-10-01,1040016,1040020,9988.0
3,2023-10-01,1040020,1040016,8649.0
4,2023-10-01,1040020,1040023,9991.0


In [38]:
df_new.tail()

,search_date,start_code,end_code,search
7275,2024-03-31,1800091,1800096,3214.0
7276,2024-03-31,1800096,1800091,2140.0
7277,2024-03-31,1800096,1800106,3200.0
7278,2024-03-31,1800106,1800096,2148.0
7279,2024-03-31,1800106,1800111,3148.0


In [45]:
# ソート（時間指定あり）
# df_new = df_new.sort_values(
#     ['start_code', 'end_code', 'passing_time']
# ).reset_index(drop=True)

# ソート（時間指定なし）
df_new = df_new.sort_values(
    ['search_date', 'start_code', 'end_code']
).reset_index(drop=True)

In [46]:
# データフレーム長が一致することを検証
assert len(df1) + len(df2) == len(df_new)

In [47]:
# 時間指定あり
# file_out = f'./search_count/search-count_specified_20231001-20240331.csv'

# 時間指定なし
file_out = f'./search_count/search-count_unspecified_20231001-20240331.csv'

In [48]:
%time df_new.to_csv(file_out, index=False)

CPU times: user 45.6 ms, sys: 7.37 ms, total: 52.9 ms
Wall time: 51.8 ms
